In [33]:
from toolz import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
%matplotlib widget


import pathlib
from lenses import lens

from collections import Counter

import re
import nltk
import pymorphy2
import fasttext.util

import tensorflow as tf

In [35]:
nltk.download('popular')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger_ru')
nltk.download('tagsets')
nltk.download('stopwords')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\aakomlev\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\aakomlev\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\aakomlev\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\aakomlev\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\aakomlev\AppData\Roaming\nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to

True

In [36]:
lmap = compose(list, map)
ltake = compose(list, take)

In [37]:
def map_df_ingredients_element(func, df):
    df = df.copy()
    df['ingredients'] = df['ingredients'].map(lambda v: lmap(func, v))
    return df

In [38]:
data_path = pathlib.Path('data')

## Загружаем датафрейм с данными

In [39]:
df = pd.read_json(data_path.joinpath('ready_dataframe.json'))
df = map_df_ingredients_element(compose(str.lower, str.strip), df)
df

,id,title,course,cuisine,ingredients
0,28195,Банановое мороженое с корицей,Выпечка и десерты,Карибская кухня,"[бананы, лимонный сок, молотая корица, ванильн..."
1,28141,Ананасово-кокосовый шербет,Выпечка и десерты,Карибская кухня,"[сахар, кокосовое молоко, корень имбиря, анана..."
2,27929,"Десерт из манго, клубники и текилы",Выпечка и десерты,Карибская кухня,"[клубника, манго, сахар, тертая цедра лайма, т..."
3,28192,Ананас с соусом из манго и рома,Выпечка и десерты,Карибская кухня,"[манго, темный ром, сок лайма, сахар, тертая ц..."
4,28463,Клубничный соус с текилой,Выпечка и десерты,Карибская кухня,"[клубника, сахарная пудра, сок лайма, текила, ..."
...,...,...,...,...,...
41433,43380,Постный борщ с фасолью,Супы,Украинская кухня,"[белая фасоль, овощной бульон, свекла, картофе..."
41434,80446,Суп энгамат,Супы,Шведская кухня,"[цветная капуста, морковь, картофель, лук-поре..."
41435,136820,Гороховый суп с блинчиками,Супы,Шведская кухня,"[горох, репчатый лук, гвоздика, свиная рулька,..."
41436,18014,Суп из брокколи и кростини с сыром бри,Супы,Шведская кухня,"[оливковое масло, репчатый лук, овощной бульон..."


## Работа с ингридиентами

Удобнее работать с листом листов:

In [40]:
ingredients = list(df['ingredients'])
# TODO !!!!!!!!! Временно берем только первые несколько записей 
ingredients = ingredients[:20]
ingredients[:3]

[['бананы', 'лимонный сок', 'молотая корица', 'ванильное мороженое'],
 ['сахар', 'кокосовое молоко', 'корень имбиря', 'ананас', 'сок лайма'],
 ['клубника',
  'манго',
  'сахар',
  'тертая цедра лайма',
  'текила',
  'сок лайма',
  'апельсиновый ликер']]

### Самые часто встречающиеся слова в ингридиентах

In [41]:
ingredients_counter = Counter()
for lst in ingredients:
    ingredients_counter.update(lst)
# first_ingresients, _ = zip(*ingredients_counter.most_common(10))
# first_ingresients = list(first_ingresients)
# first_ingresients
ingredients_counter.most_common(10)

[('сахар', 15),
 ('пшеничная мука', 14),
 ('куриное яйцо', 11),
 ('сливочное масло', 9),
 ('соль', 7),
 ('сахарная пудра', 5),
 ('сок лайма', 4),
 ('сметана', 3),
 ('сода', 3),
 ('корица', 3)]

In [42]:
morph = pymorphy2.MorphAnalyzer()
def lemmatize(word):
    return morph.parse(word)[0].normal_form

it = lmap(compose(tuple, nltk.word_tokenize), ingredients_counter)
it = lens.Each().Each().modify(lemmatize)(it)
it[:10]

[('банан',),
 ('лимонный', 'сок'),
 ('молотый', 'корица'),
 ('ванильный', 'мороженое'),
 ('сахар',),
 ('кокосовый', 'молоко'),
 ('корень', 'имбирь'),
 ('ананас',),
 ('сок', 'лайм'),
 ('клубника',)]

### Токенизация

In [43]:
ingredients_tokenized = lens.Each().Each().modify(nltk.word_tokenize)(ingredients)
ingredients_tokenized[:4]

[[['бананы'],
  ['лимонный', 'сок'],
  ['молотая', 'корица'],
  ['ванильное', 'мороженое']],
 [['сахар'],
  ['кокосовое', 'молоко'],
  ['корень', 'имбиря'],
  ['ананас'],
  ['сок', 'лайма']],
 [['клубника'],
  ['манго'],
  ['сахар'],
  ['тертая', 'цедра', 'лайма'],
  ['текила'],
  ['сок', 'лайма'],
  ['апельсиновый', 'ликер']],
 [['манго'],
  ['темный', 'ром'],
  ['сок', 'лайма'],
  ['сахар'],
  ['тертая', 'цедра', 'лайма'],
  ['ананас']]]

### Стоп-слова и фильтрация

In [44]:
stopwords = nltk.corpus.stopwords.words('russian')

In [45]:
rx = re.compile(r'[a-zA-Zа-яА-Я][a-zA-Zа-яА-Я\d\-]*')

In [46]:
ingredients_filtered = (lens
    .Each().Each().modify(
        lambda tokenized: tuple(filter(
            lambda word: word not in stopwords and rx.match(word),
            tokenized))
    )(ingredients_tokenized)
)
ingredients_filtered[:4]

[[('бананы',),
  ('лимонный', 'сок'),
  ('молотая', 'корица'),
  ('ванильное', 'мороженое')],
 [('сахар',),
  ('кокосовое', 'молоко'),
  ('корень', 'имбиря'),
  ('ананас',),
  ('сок', 'лайма')],
 [('клубника',),
  ('манго',),
  ('сахар',),
  ('тертая', 'цедра', 'лайма'),
  ('текила',),
  ('сок', 'лайма'),
  ('апельсиновый', 'ликер')],
 [('манго',),
  ('темный', 'ром'),
  ('сок', 'лайма'),
  ('сахар',),
  ('тертая', 'цедра', 'лайма'),
  ('ананас',)]]

### Лемматизация

In [47]:
ingredients_lemmatized = lens.Each().Each().Each().modify(lemmatize)(ingredients_filtered)
ingredients_lemmatized[:4]

[[('банан',),
  ('лимонный', 'сок'),
  ('молотый', 'корица'),
  ('ванильный', 'мороженое')],
 [('сахар',),
  ('кокосовый', 'молоко'),
  ('корень', 'имбирь'),
  ('ананас',),
  ('сок', 'лайм')],
 [('клубника',),
  ('манго',),
  ('сахар',),
  ('тёртый', 'цедра', 'лайм'),
  ('текила',),
  ('сок', 'лайм'),
  ('апельсиновый', 'ликёр')],
 [('манго',),
  ('тёмный', 'ром'),
  ('сок', 'лайм'),
  ('сахар',),
  ('тёртый', 'цедра', 'лайм'),
  ('ананас',)]]

In [48]:
all_words_lemmatized = Counter(lens.Each().Each().Each().collect()(ingredients_lemmatized))

all_words_lemmatized_list = list(all_words_lemmatized)
all_words_lemmatized_list.sort()
all_words_lemmatized_list[:10]
# dict(all_words.items())

['абрикосовый',
 'ананас',
 'апельсиновый',
 'банан',
 'белок',
 'брынза',
 'ванилин',
 'ваниль',
 'ванильный',
 'вишня']

In [49]:
#ОЧЕНЬ ДОЛГО
fasttext.util.download_model('ru', if_exists='ignore')

'cc.ru.300.bin'

In [50]:
ft = fasttext.load_model('cc.ru.300.bin')
dim = ft.get_dimension()
dim

300

### Получаем векторы для слов 

In [51]:
word_embeddings_dict = {}
for word in all_words_lemmatized:
    word_embeddings_dict[word] = ft.get_word_vector(word)

In [52]:
first(word_embeddings_dict.items())

('банан',
 array([ 1.94887385e-01, -6.60526380e-02,  7.88492113e-02,  9.03176740e-02,
         8.94880146e-02, -1.48379147e-01,  2.24311762e-02, -4.75879619e-03,
         1.16267549e-02,  2.68354528e-02, -3.72944064e-02,  1.64822105e-03,
         5.10738790e-02,  1.01585977e-01,  6.48163483e-02, -1.15610115e-01,
        -3.05616856e-03,  3.19645479e-02, -6.46575615e-02, -6.13481402e-02,
        -3.28286402e-02,  7.68222064e-02,  2.55825445e-02,  7.40136206e-03,
         7.12529644e-02,  2.17049755e-02,  4.97756004e-02, -8.80596880e-03,
        -4.95500583e-03,  5.70715070e-02, -9.71776620e-02,  7.35110492e-02,
         2.72706505e-02,  3.32054943e-02, -4.71901185e-05, -1.14237860e-01,
        -2.79167667e-04, -9.27831605e-02, -6.35195374e-02, -5.84830642e-02,
         1.06558718e-01, -4.90855798e-03,  7.06979185e-02, -1.20925326e-02,
         6.83131441e-02,  5.63351624e-02, -1.07557401e-02,  7.19010383e-02,
         9.81174409e-02,  8.48399326e-02,  3.53266597e-02,  5.46764955e-02,
  

In [53]:
# word_embeddings_dict

In [54]:
# word_num = [(word, i) for i, word in enumerate(words)]
# word_num[:10]

### Векторы для ингредиентов

In [55]:
all_ingredients_list =  list(set(lens.Each().Each().collect()(ingredients_lemmatized)))
ingrediendts_numbering_dict = {ingredient: i for i, ingredient in enumerate(all_ingredients_list)}
ltake(6, ingrediendts_num.items())

NameError: name 'ingrediendts_num' is not defined

Для получения вектора для ингридиента просто суммируем векторы для каждого слова в этом ингридиенте:

In [ ]:
ingrediendts_embeddings_dict = {}
for ingr in all_ingredients_list:
    ingrediendts_embeddings_dict[ingr] = np.zeros(dim)
    for word in ingr:
#         ingrediendts_embeddings_dict[ingr] += ft.get_word_vector(word) 
        ingrediendts_embeddings_dict[ingr] += word_embeddings_dict[word] # Должно заработать, когда 
                                                                         # ингредиенты будут лемматизированны
ltake(1, ingrediendts_embeddings_dict.items())[0]


Наконец, для каждого рецепта возьмем его векторизацию как сумму векторов его ингридиентов

In [ ]:
ingredients_by_recipe_embedding = [
    sum(ingrediendts_embeddings_dict[ingredient]
        for ingredient in ingredient_list) 
    for ingredient_list in ingredients_lemmatized
]
len(ingredients_by_recipe_embedding)

## Данные для нейросети

Всевозможные метки (course):

In [ ]:
all_courses = {name: num for num, name in enumerate(df['course'][~df['course'].duplicated()])}
all_courses

In [ ]:
df_for_learning = pd.DataFrame(data={
    'id': df['id'][:20],
    'ingredient_embedding': ingredients_by_recipe_embedding,
    'course_mark': [all_courses[course] for course in df['course']][:20]
})
df_for_learning

In [ ]:
# tf.keras.preprocessing.text.one_hot(all_courses, len(all_courses))


In [58]:
#Эта матрица будет использоваться для слоя Embedding в keras
ingrediendts_embeddings_matrix = np.zeros((len(ingrediendts_num), dim))
ingrediendts_embeddings_matrix.shape


(72, 300)

## Прочее:

In [20]:
it = df.groupby('cuisine').count()
it.sort_values('title', ascending=False).head(30)

,id,title,course,ingredients
cuisine,,,,
Европейская кухня,11699,11699,11699,11699
Русская кухня,7111,7111,7111,7111
Итальянская кухня,4943,4943,4943,4943
Французская кухня,3055,3055,3055,3055
Американская кухня,2192,2192,2192,2192
Авторская кухня,1822,1822,1822,1822
Китайская кухня,840,840,840,840
Греческая кухня,640,640,640,640
Мексиканская кухня,583,583,583,583


In [44]:
a = ft.get_word_vector('кокосовый')-ft.get_word_vector('кокосовое')+ft.get_word_vector('кремовое')-ft.get_word_vector('кремовый')
a@a

0.5043752